In [21]:
from pydrake.examples.acrobot import (AcrobotGeometry, AcrobotInput,
                                      AcrobotPlant, AcrobotState, AcrobotParams)
from pydrake.all import Linearize, LinearQuadraticRegulator, SymbolicVectorSystem, Variable, Saturation, \
WrapToSystem, Simulator, Polynomial

from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import LogVectorOutput

from IPython.display import SVG, display
import pydot
import numpy as np
import math
import control

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

import pydrake.forwarddiff as pf
import time

from IPython.display import display, Math, Markdown
from pydrake.examples.pendulum import PendulumPlant
from pydrake.examples.acrobot import AcrobotPlant
from pydrake.all import MultibodyPlant, Parser, SinCos, MakeVectorVariable, ToLatex, Substitute, Box
from underactuated import FindResource
from underactuated.scenarios import AddShape

from pydrake.all import (Jacobian, MathematicalProgram, Polynomial,
                         RegionOfAttraction, RegionOfAttractionOptions, Solve,
                         SymbolicVectorSystem, ToLatex, Variable, Variables,
                         plot_sublevelset_expression, AddMultibodyPlantSceneGraph, SolverOptions, 
                         CommonSolverOption)

from pydrake.solvers.mosek import MosekSolver
import sympy as sp
from sympy.physics.vector import dynamicsymbols as dynamicsymbols

from pydrake.all import (LinearQuadraticRegulator, MathematicalProgram, Variables,
                         Solve, RealContinuousLyapunovEquation)

In [17]:
from pydrake.examples.pendulum import PendulumParams
from pydrake.systems.controllers import LinearQuadraticRegulator
import scipy

In [3]:
p = PendulumParams()

In [4]:
b = p.damping()
m = p.mass()
l = p.length()
g = p.gravity()

In [9]:
A = np.zeros([3, 3])
A[0,2] = -1
A[2,0] = -g/l
A[2,2] = -b/(m*l**2)

B = np.zeros([3, 1])
B[2,0] = 1

F = np.zeros([1, 3])
F[0, 1] = -2

In [13]:
Q = np.diag([10, 0, 10])
R = [1]

In [14]:
controller = LinearQuadraticRegulator(A=A, B=B, Q=Q, R=R, F=F)

In [16]:
controller

(array([[-39.4932081 ,   0.        ,   9.04173799]]),
 array([[195.48562377,   0.        , -39.4932081 ],
        [  0.        ,   0.        ,   0.        ],
        [-39.4932081 ,   0.        ,   9.04173799]]))

In [18]:
A_cloop = A - B@controller[0]

In [19]:
A_cloop

array([[ 0.        ,  0.        , -1.        ],
       [ 0.        ,  0.        ,  0.        ],
       [19.8732081 ,  0.        , -9.44173799]])

In [20]:
P_proj = scipy.linalg.null_space(F)

In [27]:
P = RealContinuousLyapunovEquation(P_proj.T@A_cloop@P_proj, np.eye(2))

In [36]:
P = P_proj@P@P_proj.T

In [30]:
F

array([[ 0., -2.,  0.]])

In [37]:
prog = MathematicalProgram()

x = prog.NewContinuousVariables(3, 'x')

f1 = x[1]*x[2]
f2 = -x[0]*x[2]
f3 = (-b/(m*l**2))*x[2] - (g/l)*x[0] - controller[0]@x

f = np.hstack([f1, f2, f3])

V = x.T@P@x
Vdot = V.Jacobian(x).dot(f)

lambda_ = prog.NewSosPolynomial(Variables(x), 2)
lambda_s = prog.NewFree

/tmp/ipykernel_10052/3870630733.py:7: RuntimeWarning: invalid value encountered in matmul
  f3 = (-b/(m*l**2))*x[2] - (g/l)*x[0] - controller[0]@x
/tmp/ipykernel_10052/3870630733.py:7: RuntimeWarning: invalid value encountered in subtract
  f3 = (-b/(m*l**2))*x[2] - (g/l)*x[0] - controller[0]@x
/tmp/ipykernel_10052/3870630733.py:11: RuntimeWarning: invalid value encountered in matmul
  V = x.T@P@x


In [35]:
P

array([[ 1.34291837, -0.0251595 ],
       [-0.0251595 ,  0.05562106]])